# IBM Watson OpenScale Lab instructions

This notebook should be run in a Watson Studio project, using with **Python 3.5 with Spark** runtime environment. **If you are viewing this in Watson Studio and do not see Python 3.5 with Spark in the upper right corner of your screen, please update the runtime now.** It requires service credentials for the following Cloud services:
  * IBM Watson OpenScale
  * Watson Machine Learning
  
If you have a paid Cloud account, you may also provision a **Databases for PostgreSQL** or **Db2 Warehouse** service to take full advantage of integration with Watson Studio and continuous learning services. If you choose not to provision this paid service, you can use the free internal PostgreSQL storage with OpenScale, but will not be able to configure continuous learning for your model.

The notebook will train, create and deploy a Propensity to Buy Model, configure OpenScale to monitor that deployment, and inject seven days' worth of historical records and measurements for viewing in the OpenScale Insights dashboard.

# Package installation

In [ ]:
!rm -rf $PIP_BUILD
!pip install psycopg2-binary | tail -n 1
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install --upgrade numpy --no-cache | tail -n 1
!pip install --upgrade lime --no-cache | tail -n 1
!pip install --upgrade SciPy --no-cache | tail -n 1

# Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/ai-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

In [41]:
CLOUD_API_KEY = "Ize6GGd8eXW1CRPq_JDemA0KeFHbOLrjk9JiAPOMglbQ"

Next you will need credentials for Watson Machine Learning. If you already have a WML instance, you may use credentials for it. To provision a new Lite instance of WML, use the [Cloud catalog](https://cloud.ibm.com/catalog/services/machine-learning), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your WML credentials into the cell below.

In [42]:
WML_CREDENTIALS ={
  "apikey": "wcPASQdXvtEPgA79lY4sYpCfiWAOoSNudy0hh24PMNbb",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/97deeb0b7e78431438a00a04f20580b7:e7e6948c-829b-4d59-ae09-3f991dfe52cf::",
  "iam_apikey_name": "auto-generated-apikey-f4fafb37-2a2d-477c-a9c6-0ff5400609f6",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/97deeb0b7e78431438a00a04f20580b7::serviceid:ServiceId-8e03beb2-f9d6-4351-b0e5-2add2a3437df",
  "instance_id": "e7e6948c-829b-4d59-ae09-3f991dfe52cf",
  "password": "d3538d26-7a6c-4e2f-b114-6a02041ce00b",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "f4fafb37-2a2d-477c-a9c6-0ff5400609f6"
}

This lab can use Databases for PostgreSQL, Db2 Warehouse, or a free internal verison of PostgreSQL to create a datamart for OpenScale.

If you have previously configured OpenScale, it will use your existing datamart, and not interfere with any models you are currently monitoring. Do not update the cell below.

If you do not have a paid Cloud account or would prefer not to provision this paid service, you may use the free internal PostgreSQL service with OpenScale. Do not update the cell below.

To provision a new instance of Db2 Warehouse, locate [Db2 Warehouse in the Cloud catalog](https://cloud.ibm.com/catalog/services/db2-warehouse), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Db2 Warehouse credentials into the cell below.

To provision a new instance of Databases for PostgreSQL, locate [Databases for PostgreSQL in the Cloud catalog](https://cloud.ibm.com/catalog/services/databases-for-postgresql), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Databases for PostgreSQL credentials into the cell below.

In [43]:
DB_CREDENTIALS = None

__If you previously configured OpenScale to use the free internal version of PostgreSQL, you can switch to a new datamart using a paid database service.__ If you would like to delete the internal PostgreSQL configuration and create a new one using service credentials supplied in the cell above, set the __KEEP_MY_INTERNAL_POSTGRES__ variable below to __False__ below. In this case, the notebook will remove your existing internal PostgreSQL datamart and create a new one with the supplied credentials. __*NO DATA MIGRATION WILL OCCUR.*__

In [44]:
KEEP_MY_INTERNAL_POSTGRES = True

# Run the notebook

At this point, the notebook is ready to run. You can either run the cells one at a time, or click the **Kernel** option above and select **Restart and Run All** to run all the cells.

# Load and explore data

## Load the training data from github

In [45]:
!rm df_training.csv
!wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/df_training.csv

--2019-03-07 16:02:18--  https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/df_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 301618 (295K) [text/plain]
Saving to: 'df_training.csv'

df_training.csv     100%[===================>] 294.55K  --.-KB/s    in 0.01s   

2019-03-07 16:02:18 (25.0 MB/s) - 'df_training.csv' saved [301618/301618]



In [46]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("df_training.csv", sep=",", header=0)
df_data = spark.read.csv(path="df_training.csv", sep=",", header=True, inferSchema=True)
df_data.head()

Row(INCOME=125000.0075, AGE_IN_YEARS=42.394906, HAS_CHILDREN=0.0, RENTER=0, OWNER=1, GOOD_CREDIT=1, COLLEGE_DEGREE=0, AVG_DIST_FROM_HOME=0.0, MILLENIAL=0, MID_LEVEL_AUTO=0, DISTANCE_FROM_HOME=9, TRAVELER=0, TIME_OF_DAY=13, BUY_LABEL='NOT_BUY')

## Explore data

In [47]:
df_data.printSchema()

root
 |-- INCOME: double (nullable = true)
 |-- AGE_IN_YEARS: double (nullable = true)
 |-- HAS_CHILDREN: double (nullable = true)
 |-- RENTER: integer (nullable = true)
 |-- OWNER: integer (nullable = true)
 |-- GOOD_CREDIT: integer (nullable = true)
 |-- COLLEGE_DEGREE: integer (nullable = true)
 |-- AVG_DIST_FROM_HOME: double (nullable = true)
 |-- MILLENIAL: integer (nullable = true)
 |-- MID_LEVEL_AUTO: integer (nullable = true)
 |-- DISTANCE_FROM_HOME: integer (nullable = true)
 |-- TRAVELER: integer (nullable = true)
 |-- TIME_OF_DAY: integer (nullable = true)
 |-- BUY_LABEL: string (nullable = true)



In [48]:
print("Number of records: " + str(df_data.count()))

Number of records: 4710


# Create a model

In [49]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

MODEL_NAME = "Propensity to Buy"
DEPLOYMENT_NAME = "Propensity to Buy"

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

spark_df.printSchema()

Number of records for training: 3787
Number of records for evaluation: 923
root
 |-- INCOME: double (nullable = true)
 |-- AGE_IN_YEARS: double (nullable = true)
 |-- HAS_CHILDREN: double (nullable = true)
 |-- RENTER: integer (nullable = true)
 |-- OWNER: integer (nullable = true)
 |-- GOOD_CREDIT: integer (nullable = true)
 |-- COLLEGE_DEGREE: integer (nullable = true)
 |-- AVG_DIST_FROM_HOME: double (nullable = true)
 |-- MILLENIAL: integer (nullable = true)
 |-- MID_LEVEL_AUTO: integer (nullable = true)
 |-- DISTANCE_FROM_HOME: integer (nullable = true)
 |-- TRAVELER: integer (nullable = true)
 |-- TIME_OF_DAY: integer (nullable = true)
 |-- BUY_LABEL: string (nullable = true)



In [50]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model


In [51]:
si_Label = StringIndexer(inputCol="BUY_LABEL", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

In [52]:
va_features = VectorAssembler(inputCols=["INCOME", "AGE_IN_YEARS", "HAS_CHILDREN", "RENTER", "OWNER", "GOOD_CREDIT", "COLLEGE_DEGREE", \
                                         "AVG_DIST_FROM_HOME","MILLENIAL","MID_LEVEL_AUTO","DISTANCE_FROM_HOME","TRAVELER","TIME_OF_DAY"], outputCol="features")

In [53]:
from pyspark.ml.classification import RandomForestClassifier
classifier = RandomForestClassifier(featuresCol="features")

pipeline = Pipeline(stages=[ si_Label, va_features, classifier, label_converter])
model = pipeline.fit(train_data)

In [54]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve)

areaUnderROC = 0.80743


# Save and deploy the model

In [55]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

### Remove existing model and deployment

In [56]:
model_deployment_ids = wml_client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = wml_client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

Deleting deployment id 4a02795f-d526-4f93-b16a-2c427472cd41
Deleting model id 981ca5e9-b5d3-40fb-84d3-05d0b7283ee1
----  ----  -------  ---------
GUID  NAME  CREATED  FRAMEWORK
----  ----  -------  ---------


In [57]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(MODEL_NAME),
    wml_client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    wml_client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "areaUnderROC",
           "value": area_under_curve,
           "threshold": 0.8
        }
    ]
}

In [58]:
wml_models = wml_client.repository.get_details()
model_uid = None
for model_in in wml_models['models']['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break

if model_uid is None:
    print("Storing model ...")

    published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline)
    model_uid = wml_client.repository.get_model_uid(published_model_details)
    print("Done")

Storing model ...
Done


In [59]:
model_uid

'123412b9-235c-4441-8ab4-46b67f6b5aa4'

In [60]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break

if deployment_uid is None:
    print("Deploying model...")

    deployment = wml_client.deployments.create(artifact_uid=model_uid, name=DEPLOYMENT_NAME, asynchronous=False)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

Deploying model...


#######################################################################################

Synchronous deployment creation for uid: '123412b9-235c-4441-8ab4-46b67f6b5aa4' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='90feae8c-ad4f-4f78-83d5-1d2e81b9c578'
------------------------------------------------------------------------------------------------


Model id: 123412b9-235c-4441-8ab4-46b67f6b5aa4
Deployment id: 90feae8c-ad4f-4f78-83d5-1d2e81b9c578


# Configure OpenScale

In [61]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

### Get AI OpenScale GUID

In [62]:
import requests

AIOS_GUID = None
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': CLOUD_API_KEY
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

resources = json.loads(requests.get('https://resource-controller.cloud.ibm.com/v2/resource_instances', headers=iam_headers).text)['resources']
for resource in resources:
    if "aiopenscale" in resource['id'].lower():
        AIOS_GUID = resource['guid']
        
AIOS_CREDENTIALS = {
    "instance_guid": AIOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if AIOS_GUID is None:
    print('AI OpenScale GUID NOT FOUND')
else:
    print(AIOS_GUID)

95cf4deb-3802-49fb-bf0d-4ee0a9f6ac9e


## Create schema and datamart

In [63]:
ai_client = APIClient(aios_credentials=AIOS_CREDENTIALS)
ai_client.version

'2.1.1'

### Set up datamart

In [64]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    if 'internal_database' in data_mart_details and data_mart_details['internal_database']:
        if KEEP_MY_INTERNAL_POSTGRES:
            print('Using existing internal datamart.')
        else:
            if DB_CREDENTIALS is None:
                print('No postgres credentials supplied. Using existing internal datamart')
            else:
                print('Switching to external datamart')
                ai_client.data_mart.delete(force=True)
                ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    else:
        print('Using existing external datamart')
except:
    if DB_CREDENTIALS is None:
        print('Setting up internal datamart')
        ai_client.data_mart.setup(internal_db=True)
    else:
        print('Setting up external datamart')
        ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    

Setting up internal datamart


In [65]:
data_mart_details = ai_client.data_mart.get_details()
data_mart_details

{'database_configuration': {},
 'internal_database': True,
 'service_instance_crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/97deeb0b7e78431438a00a04f20580b7:95cf4deb-3802-49fb-bf0d-4ee0a9f6ac9e::',
 'status': {'state': 'active'}}

## Bind machine learning engines

In [66]:
binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance(WML_CREDENTIALS))
if binding_uid is None:
    binding_uid = ai_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
bindings_details = ai_client.data_mart.bindings.get_details()
ai_client.data_mart.bindings.list()

e7e6948c-829b-4d59-ae09-3f991dfe52cf,WML instance,watson_machine_learning,2019-03-07T16:02:52.484Z


In [67]:
print(binding_uid)

e7e6948c-829b-4d59-ae09-3f991dfe52cf


In [68]:
ai_client.data_mart.bindings.list_assets()

123412b9-235c-4441-8ab4-46b67f6b5aa4,Propensity to Buy,2019-03-07T16:02:41.068Z,model,mllib-2.3,e7e6948c-829b-4d59-ae09-3f991dfe52cf,False


## Subscriptions

### Remove existing  propensity to buy subscriptions

In [69]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == MODEL_NAME:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', MODEL_NAME)

In [70]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='BUY_LABEL',
    prediction_column='predictedLabel',
    probability_column='probability',
    feature_columns = ["INCOME", "AGE_IN_YEARS", "HAS_CHILDREN", "RENTER", "OWNER", "GOOD_CREDIT", "COLLEGE_DEGREE","AVG_DIST_FROM_HOME",
                       "MILLENIAL","MID_LEVEL_AUTO","DISTANCE_FROM_HOME","TRAVELER","TIME_OF_DAY"],
    categorical_columns = ["HAS_CHILDREN", "RENTER", "OWNER", "GOOD_CREDIT", "COLLEGE_DEGREE","MILLENIAL","MID_LEVEL_AUTO","TRAVELER"]
))

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            subscription = ai_client.data_mart.subscriptions.get(sub)

Get subscription list

In [71]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

123412b9-235c-4441-8ab4-46b67f6b5aa4,Propensity to Buy,model,e7e6948c-829b-4d59-ae09-3f991dfe52cf,2019-03-07T16:03:00.303Z


In [72]:
subscription.get_details()

{'entity': {'asset': {'asset_id': '123412b9-235c-4441-8ab4-46b67f6b5aa4',
   'asset_type': 'model',
   'created_at': '2019-03-07T16:02:41.068Z',
   'name': 'Propensity to Buy',
   'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/e7e6948c-829b-4d59-ae09-3f991dfe52cf/published_models/123412b9-235c-4441-8ab4-46b67f6b5aa4'},
  'asset_properties': {'categorical_fields': ['HAS_CHILDREN',
    'RENTER',
    'OWNER',
    'GOOD_CREDIT',
    'COLLEGE_DEGREE',
    'MILLENIAL',
    'MID_LEVEL_AUTO',
    'TRAVELER'],
   'feature_fields': ['INCOME',
    'AGE_IN_YEARS',
    'HAS_CHILDREN',
    'RENTER',
    'OWNER',
    'GOOD_CREDIT',
    'COLLEGE_DEGREE',
    'AVG_DIST_FROM_HOME',
    'MILLENIAL',
    'MID_LEVEL_AUTO',
    'DISTANCE_FROM_HOME',
    'TRAVELER',
    'TIME_OF_DAY'],
   'input_data_schema': {'fields': [{'metadata': {'modeling_role': 'feature'},
      'name': 'INCOME',
      'nullable': True,
      'type': 'double'},
     {'metadata': {'modeling_role': 'feature'},
      'name': 

### Score the model so we can configure monitors

In [73]:
propensity_to_buy_scoring_endpoint = None
print(deployment_uid)

for deployment in wml_client.deployments.get_details()['resources']:
    if deployment_uid in deployment['metadata']['guid']:
        propensity_to_buy_scoring_endpoint = deployment['entity']['scoring_url']
        
print(propensity_to_buy_scoring_endpoint)

90feae8c-ad4f-4f78-83d5-1d2e81b9c578
https://us-south.ml.cloud.ibm.com/v3/wml_instances/e7e6948c-829b-4d59-ae09-3f991dfe52cf/deployments/90feae8c-ad4f-4f78-83d5-1d2e81b9c578/online


In [74]:
fields = ["INCOME", "AGE_IN_YEARS", "HAS_CHILDREN", "RENTER","OWNER", "GOOD_CREDIT", "COLLEGE_DEGREE", "AVG_DIST_FROM_HOME","MILLENIAL", "MID_LEVEL_AUTO",
          "DISTANCE_FROM_HOME", "TRAVELER", "TIME_OF_DAY"]
values = [
[87500.0048,82.728962,0.0,0,1,1,0,9.16,0,1,3,0,18],
[47500.0006,31.970484,0.0,0,1,1,1,14.7,1,1,3,0,12],
[250000.007,61.898855,0.0,0,1,1,1,13.5,0,1,9,0,18],
[17500.0087,79.146277,0.0,0,1,1,0,0.11,0,1,1,0,11],
[125000.0015,70.388906,0.0,0,1,1,1,6.76,0,0,1,0,15],
[62500.0075,36.73303,1.0,0,1,1,0,22.86,0,0,13,0,10],
[87500.0018,65.726646,1.0,0,1,1,0,26.65,0,0,18,0,13],
[87500.0046,63.057304,0.0,0,1,1,0,13.86,0,1,2,0,6],
[62500.0085,57.147446,1.0,0,1,0,0,43.54,0,1,31,0,18],
[17500.0053,60.14493,0.0,0,1,1,0,3.1,0,1,9,0,12]
]

payload_scoring = {"fields": fields,"values": values}
scoring_response = wml_client.deployments.score(propensity_to_buy_scoring_endpoint, payload_scoring)

print(scoring_response)

{'fields': ['INCOME', 'AGE_IN_YEARS', 'HAS_CHILDREN', 'RENTER', 'OWNER', 'GOOD_CREDIT', 'COLLEGE_DEGREE', 'AVG_DIST_FROM_HOME', 'MILLENIAL', 'MID_LEVEL_AUTO', 'DISTANCE_FROM_HOME', 'TRAVELER', 'TIME_OF_DAY', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [[87500.0048, 82.728962, 0.0, 0, 1, 1, 0, 9.16, 0, 1, 3, 0, 18, [87500.0048, 82.728962, 0.0, 0.0, 1.0, 1.0, 0.0, 9.16, 0.0, 1.0, 3.0, 0.0, 18.0], [2.27943140086085, 17.720568599139153], [0.11397157004304248, 0.8860284299569575], 1.0, 'NOT_BUY'], [47500.0006, 31.970484, 0.0, 0, 1, 1, 1, 14.7, 1, 1, 3, 0, 12, [47500.0006, 31.970484, 0.0, 0.0, 1.0, 1.0, 1.0, 14.7, 1.0, 1.0, 3.0, 0.0, 12.0], [2.691699757759864, 17.308300242240136], [0.1345849878879932, 0.8654150121120068], 1.0, 'NOT_BUY'], [250000.007, 61.898855, 0.0, 0, 1, 1, 1, 13.5, 0, 1, 9, 0, 18, [250000.007, 61.898855, 0.0, 0.0, 1.0, 1.0, 1.0, 13.5, 0.0, 1.0, 9.0, 0.0, 18.0], [5.047110922304637, 14.952889077695362], [0.25235554611523187, 0.7476

## Quality and feedback monitoring

### Enable quality monitoring

Wait ten seconds to allow the payload logging table to be set up before we begin enabling monitors.

In [75]:
time.sleep(10)
subscription.quality_monitoring.enable(threshold=0.7, min_records=50)

### Feedback logging

In [76]:
!rm df_feedback.json
!wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/df_feedback.json

rm: cannot remove 'df_feedback.json': No such file or directory
--2019-03-07 16:03:14--  https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/df_feedback.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 294045 (287K) [text/plain]
Saving to: 'df_feedback.json'

df_feedback.json    100%[===================>] 287.15K  --.-KB/s    in 0.01s   

2019-03-07 16:03:14 (25.6 MB/s) - 'df_feedback.json' saved [294045/294045]



In [77]:
with open('df_feedback.json') as feedback_file:
    df_feedback = json.load(feedback_file)
subscription.feedback_logging.store(df_feedback['data'])

In [78]:
subscription.feedback_logging.show_table()

87500.0083,86.814597,1.0,0,1,1,0,0.0,0,0,9,0,15,NOT_BUY,2019-03-07 16:03:15.993000+00:00
87500.0078,50.731962,1.0,0,1,1,1,0.0,0,0,6,0,6,NOT_BUY,2019-03-07 16:03:15.993000+00:00
87500.0075,77.565337,0.0,0,0,1,1,0.05,0,0,6,0,14,NOT_BUY,2019-03-07 16:03:15.993000+00:00
62500.0052,79.309786,1.0,0,1,1,1,0.14,0,1,7,0,18,NOT_BUY,2019-03-07 16:03:15.993000+00:00
70000.0037,73.643673,0.0,0,1,1,0,0.16,0,0,3,0,14,NOT_BUY,2019-03-07 16:03:15.993000+00:00
125000.0013,58.96913,0.0,0,1,1,1,0.16,0,0,9,0,11,NOT_BUY,2019-03-07 16:03:15.993000+00:00
80372.1848753994,55.4524022178,0.0,0,0,1,0,0.16,0,0,8,0,7,NOT_BUY,2019-03-07 16:03:15.993000+00:00
42500.0011,99.806713,0.0,0,1,1,0,0.17,0,1,5,0,6,NOT_BUY,2019-03-07 16:03:15.993000+00:00
70000.0097,47.147277,1.0,0,1,1,0,0.18,0,1,8,0,10,NOT_BUY,2019-03-07 16:03:15.993000+00:00
87500.0003,35.890786,1.0,0,1,1,0,0.18,0,0,137,1,11,BUY,2019-03-07 16:03:15.993000+00:00


In [79]:
run_details = subscription.quality_monitoring.run()
status = run_details['status']
id = run_details['id']
print(id)

print("Run status: {}".format(status))

start_time = time.time()
elapsed_time = 0

while status != 'completed' and elapsed_time < 60:
    time.sleep(10)
    run_details = subscription.quality_monitoring.get_run_details(run_uid=id)
    status = run_details['status']
    elapsed_time = time.time() - start_time
    print("Run status: {}".format(status))

8eb04535-b05a-404d-a7f4-9d67e0ebea9f
Run status: initializing
Run status: running
Run status: completed


In [80]:
subscription.quality_monitoring.get_run_details()

{'evaluations': [{'completed_at': '2019-03-07T16:03:36.125Z',
   'data_mart_id': '95cf4deb-3802-49fb-bf0d-4ee0a9f6ac9e',
   'flags': {'batch_size': 1000,
    'fetch_size': 1000,
    'parallelism': 4,
    'window_in_millis': 2000},
   'id': '8eb04535-b05a-404d-a7f4-9d67e0ebea9f',
   'metrics': {'areaUnderPR': 0.7014431772486022,
    'areaUnderROC': 0.8201962492313999},
   'problem_type': 'binary',
   'rows_in_range': 4748,
   'rows_max': 10000,
   'rows_min': 50,
   'service_binding_id': 'e7e6948c-829b-4d59-ae09-3f991dfe52cf',
   'stages': [{'completed_at': '2019-03-07T16:03:24.804Z',
     'id': 1,
     'name': 'Prerequisite check',
     'properties': {'input_columns': ['INCOME',
       'AGE_IN_YEARS',
       'HAS_CHILDREN',
       'RENTER',
       'OWNER',
       'GOOD_CREDIT',
       'COLLEGE_DEGREE',
       'AVG_DIST_FROM_HOME',
       'MILLENIAL',
       'MID_LEVEL_AUTO',
       'DISTANCE_FROM_HOME',
       'TRAVELER',
       'TIME_OF_DAY'],
      'training_columns': ['INCOME',
    

In [81]:
subscription.quality_monitoring.show_table()

2019-03-07 16:03:24.610000+00:00,0.8201962492313999,0.7,e7e6948c-829b-4d59-ae09-3f991dfe52cf,123412b9-235c-4441-8ab4-46b67f6b5aa4,90feae8c-ad4f-4f78-83d5-1d2e81b9c578,Accuracy_evaluation_8eb04535-b05a-404d-a7f4-9d67e0ebea9f,


In [82]:
subscription.quality_monitoring._get_data_from_rest_api()

[[datetime.datetime(2019, 3, 7, 16, 3, 24, 610000, tzinfo=tzlocal()),
  0.8201962492313999,
  0.7,
  'e7e6948c-829b-4d59-ae09-3f991dfe52cf',
  '123412b9-235c-4441-8ab4-46b67f6b5aa4',
  '90feae8c-ad4f-4f78-83d5-1d2e81b9c578',
  'Accuracy_evaluation_8eb04535-b05a-404d-a7f4-9d67e0ebea9f',
  '']]

In [83]:
ai_client.data_mart.get_deployment_metrics()

{'deployment_metrics': [{'asset': {'asset_id': '123412b9-235c-4441-8ab4-46b67f6b5aa4',
    'asset_type': 'model',
    'created_at': '2019-03-07T16:02:41.068Z',
    'name': 'Propensity to Buy',
    'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/e7e6948c-829b-4d59-ae09-3f991dfe52cf/published_models/123412b9-235c-4441-8ab4-46b67f6b5aa4'},
   'deployment': {'created_at': '2019-03-07T16:02:41.098Z',
    'deployment_id': '90feae8c-ad4f-4f78-83d5-1d2e81b9c578',
    'deployment_rn': '',
    'deployment_type': 'online',
    'name': 'Propensity to Buy',
    'scoring_endpoint': {'request_headers': {'Content-Type': 'application/json'},
     'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/e7e6948c-829b-4d59-ae09-3f991dfe52cf/deployments/90feae8c-ad4f-4f78-83d5-1d2e81b9c578/online'},
    'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/e7e6948c-829b-4d59-ae09-3f991dfe52cf/deployments/90feae8c-ad4f-4f78-83d5-1d2e81b9c578'},
   'metrics': [{'issues': 0,
     'metric_ty

## Fairness monitoring

In [84]:
subscription.fairness_monitoring.enable(
            features=[
                Feature("MID_LEVEL_AUTO", majority=[[0,0]], minority=[[1,1]], threshold=0.80)
            ],
            favourable_classes=['BUY'],
            unfavourable_classes=['NOT_BUY'],
            min_records=1000,
            training_data=pd_data
        )

## Score the model again now that monitoring is configured

In [85]:
!rm df_payload_scoring.json
!wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/df_payload_scoring.json

rm: cannot remove 'df_payload_scoring.json': No such file or directory
--2019-03-07 16:03:47--  https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/df_payload_scoring.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256874 (251K) [text/plain]
Saving to: 'df_payload_scoring.json'

df_payload_scoring. 100%[===================>] 250.85K  --.-KB/s    in 0.009s  

2019-03-07 16:03:47 (26.6 MB/s) - 'df_payload_scoring.json' saved [256874/256874]



Score 1000 randomly chosen records

In [86]:
import random

with open('df_payload_scoring.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(1000):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(propensity_to_buy_scoring_endpoint, payload_scoring)
print(scoring_response)

{'fields': ['INCOME', 'AGE_IN_YEARS', 'HAS_CHILDREN', 'RENTER', 'OWNER', 'GOOD_CREDIT', 'COLLEGE_DEGREE', 'AVG_DIST_FROM_HOME', 'MILLENIAL', 'MID_LEVEL_AUTO', 'DISTANCE_FROM_HOME', 'TRAVELER', 'TIME_OF_DAY', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [[250000.0012, 55.4452022178, 1.0, 0, 1, 1, 0, 25.92, 0, 0, 22, 0, 11, [250000.0012, 55.4452022178, 1.0, 0.0, 1.0, 1.0, 0.0, 25.92, 0.0, 0.0, 22.0, 0.0, 11.0], [16.802449988331507, 3.197550011668495], [0.8401224994165754, 0.15987750058342476], 0.0, 'BUY'], [125000.0061, 45.897955, 1.0, 0, 0, 0, 0, 0.54, 0, 0, 8, 0, 18, [13, [0, 1, 2, 7, 10, 12], [125000.0061, 45.897955, 1.0, 0.54, 8.0, 18.0]], [9.341790726936395, 10.658209273063603], [0.46708953634681977, 0.5329104636531802], 1.0, 'NOT_BUY'], [70000.0078, 47.564268, 1.0, 1, 0, 1, 1, 36.72, 0, 1, 19, 0, 9, [70000.0078, 47.564268, 1.0, 1.0, 0.0, 1.0, 1.0, 36.72, 0.0, 1.0, 19.0, 0.0, 9.0], [17.87975596482749, 2.120244035172512], [0.8939877982413744,

In [87]:
payload_scoring

{'fields': ['INCOME',
  'AGE_IN_YEARS',
  'HAS_CHILDREN',
  'RENTER',
  'OWNER',
  'GOOD_CREDIT',
  'COLLEGE_DEGREE',
  'AVG_DIST_FROM_HOME',
  'MILLENIAL',
  'MID_LEVEL_AUTO',
  'DISTANCE_FROM_HOME',
  'TRAVELER',
  'TIME_OF_DAY'],
 'values': [[250000.0012,
   55.4452022178,
   1.0,
   0,
   1,
   1,
   0,
   25.92,
   0,
   0,
   22,
   0,
   11],
  [125000.0061, 45.897955, 1.0, 0, 0, 0, 0, 0.54, 0, 0, 8, 0, 18],
  [70000.0078, 47.564268, 1.0, 1, 0, 1, 1, 36.72, 0, 1, 19, 0, 9],
  [125000.0074, 46.97523, 1.0, 0, 1, 1, 1, 59.22, 0, 0, 34, 0, 10],
  [225000.0043, 42.728462, 1.0, 0, 0, 0, 1, 34.74, 0, 0, 39, 0, 10],
  [87500.006, 56.646657, 1.0, 0, 0, 1, 1, 18.9, 0, 0, 25, 0, 13],
  [87500.0028, 61.223881, 1.0, 0, 1, 1, 1, 18.33, 0, 0, 33, 0, 18],
  [87500.0008, 44.389575, 1.0, 0, 1, 1, 1, 2.52, 0, 1, 6, 0, 19],
  [125000.0093, 57.482264, 1.0, 0, 1, 1, 0, 14.04, 0, 0, 7, 0, 14],
  [125000.0023, 43.892786, 1.0, 0, 1, 1, 0, 20.88, 0, 0, 26, 0, 8],
  [125000.0075, 44.900039, 1.0, 0, 1, 1, 

In [88]:
subscription.get_details()

{'entity': {'asset': {'asset_id': '123412b9-235c-4441-8ab4-46b67f6b5aa4',
   'asset_type': 'model',
   'created_at': '2019-03-07T16:02:41.068Z',
   'name': 'Propensity to Buy',
   'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/e7e6948c-829b-4d59-ae09-3f991dfe52cf/published_models/123412b9-235c-4441-8ab4-46b67f6b5aa4'},
  'asset_properties': {'categorical_fields': ['HAS_CHILDREN',
    'RENTER',
    'OWNER',
    'GOOD_CREDIT',
    'COLLEGE_DEGREE',
    'MILLENIAL',
    'MID_LEVEL_AUTO',
    'TRAVELER'],
   'feature_fields': ['INCOME',
    'AGE_IN_YEARS',
    'HAS_CHILDREN',
    'RENTER',
    'OWNER',
    'GOOD_CREDIT',
    'COLLEGE_DEGREE',
    'AVG_DIST_FROM_HOME',
    'MILLENIAL',
    'MID_LEVEL_AUTO',
    'DISTANCE_FROM_HOME',
    'TRAVELER',
    'TIME_OF_DAY'],
   'input_data_schema': {'fields': [{'metadata': {'modeling_role': 'feature'},
      'name': 'INCOME',
      'nullable': True,
      'type': 'double'},
     {'metadata': {'modeling_role': 'feature'},
      'name': 

# Insert historical payloads

In [89]:
!rm payload_history*.json
!wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/payload_history_1.json
!wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/payload_history_2.json
!wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/payload_history_3.json
!wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/payload_history_4.json
!wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/payload_history_5.json
!wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/payload_history_6.json
!wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/payload_history_7.json

rm: cannot remove 'payload_history*.json': No such file or directory
--2019-03-07 16:03:49--  https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/payload_history_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2588835 (2.5M) [text/plain]
Saving to: 'payload_history_1.json'

payload_history_1.j 100%[===================>]   2.47M  --.-KB/s    in 0.07s   

2019-03-07 16:03:49 (37.8 MB/s) - 'payload_history_1.json' saved [2588835/2588835]

--2019-03-07 16:03:50--  https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/payload_history_2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2

In [90]:
historyDays = 7

In [91]:
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
import datetime
import time

for day in range(historyDays):
    print('Loading day {}'.format(day + 1))
    history_file = 'payload_history_' + str(day + 1) + '.json'
    with open(history_file) as f:
        payloads = json.load(f)
        hourly_records = int(len(payloads) / 24)
        index = 0
        for hour in range(24):
            recordsList = []
            for i in range(hourly_records):
                score_time = str(datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1))))
                recordsList.append(PayloadRecord(request=payloads[index]['request'], response=payloads[index]['response'], scoring_timestamp=score_time))
                index += 1
            subscription.payload_logging.store(records=recordsList)
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


In [92]:
data_mart_id = subscription.get_details()['metadata']['url'].split('/service_bindings')[0].split('marts/')[1]
print(data_mart_id)

95cf4deb-3802-49fb-bf0d-4ee0a9f6ac9e


In [93]:
performance_metrics_url = 'https://api.aiopenscale.cloud.ibm.com' + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/metrics'
print(performance_metrics_url)

https://api.aiopenscale.cloud.ibm.com/v1/data_marts/95cf4deb-3802-49fb-bf0d-4ee0a9f6ac9e/metrics


## Insert historical fairness metrics

In [94]:
!rm fairness_records.json
!wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/fairness_records.json

rm: cannot remove 'fairness_records.json': No such file or directory
--2019-03-07 16:04:28--  https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/fairness_records.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11380 (11K) [text/plain]
Saving to: 'fairness_records.json'

fairness_records.js 100%[===================>]  11.11K  --.-KB/s    in 0s      

2019-03-07 16:04:28 (117 MB/s) - 'fairness_records.json' saved [11380/11380]



In [95]:
import random
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

with open('fairness_records.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        qualityMetric = {
            'metric_type': 'fairness',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': random.choice(payloads)
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## Insert historical quality metrics

In [96]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

measurements = [0.84, 0.81, 0.68, 0.72, 0.80, 0.84, 0.83]
for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        qualityMetric = {
            'metric_type': 'quality',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'quality': measurements[day],
                'threshold': 0.8,
                'metrics': [
                    {
                        'name': 'auroc',
                        'value': measurements[day],
                        'threshold': 0.8
                    }
                ]
            }
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## Insert historical performance metrics

In [97]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        score_count = random.randint(600, 6000)
        score_resp = random.uniform(600, 3000)

        performanceMetric = {
            'metric_type': 'performance',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'response_time': score_resp,
                'records': score_count
            }
        }

        response = requests.post(performance_metrics_url, json=[performanceMetric], headers=iam_headers)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## Configure Explainability

In [98]:
from ibm_ai_openscale.supporting_classes import *
subscription.explainability.enable(training_data=pd_data)

In [99]:
subscription.explainability.get_details()

{'enabled': True,
 'parameters': {'training_statistics': {'base_values': {'0': 87500.0022,
    '1': 53.899755,
    '10': 16.0,
    '11': 0,
    '12': 11.0,
    '2': 0.0,
    '3': 0,
    '4': 1,
    '5': 1,
    '6': 0,
    '7': 22.32,
    '8': 0,
    '9': 0},
   'categorical_columns': ['HAS_CHILDREN',
    'RENTER',
    'OWNER',
    'GOOD_CREDIT',
    'COLLEGE_DEGREE',
    'MILLENIAL',
    'MID_LEVEL_AUTO',
    'TRAVELER'],
   'categorical_columns_encoding_mapping': {'11': [0, 1],
    '2': [0.0, 1.0],
    '3': [0, 1],
    '4': [0, 1],
    '5': [0, 1],
    '6': [0, 1],
    '8': [0, 1],
    '9': [0, 1]},
   'categorical_counts': {'11': {'0': 4546, '1': 164},
    '2': {'0.0': 2463, '1.0': 2247},
    '3': {'0': 4593, '1': 117},
    '4': {'0': 1355, '1': 3355},
    '5': {'0': 658, '1': 4052},
    '6': {'0': 3263, '1': 1447},
    '8': {'0': 4417, '1': 293},
    '9': {'0': 3677, '1': 1033}},
   'class_labels': ['NOT_BUY', 'BUY'],
   'd_bins': {'0': [80372.1766753994, 87500.0022, 125000.0031],
 

## Run fairness monitor

Kick off a fairness monitor run on current data. Depending on how fast the monitor runs, the table may not contain the most recent results.

In [100]:
run_details = subscription.fairness_monitoring.run()

In [101]:
subscription.fairness_monitoring.show_table()

2019-03-07 15:04:28+00:00,MID_LEVEL_AUTO,"[1, 1]",False,1.0,50.2,e7e6948c-829b-4d59-ae09-3f991dfe52cf,123412b9-235c-4441-8ab4-46b67f6b5aa4,123412b9-235c-4441-8ab4-46b67f6b5aa4,90feae8c-ad4f-4f78-83d5-1d2e81b9c578,


## Additional data to help debugging

In [102]:
#print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)
print('Binding:', binding_uid)
print('Scoring URL:', propensity_to_buy_scoring_endpoint)

Model: 123412b9-235c-4441-8ab4-46b67f6b5aa4
Deployment: 90feae8c-ad4f-4f78-83d5-1d2e81b9c578
Binding: e7e6948c-829b-4d59-ae09-3f991dfe52cf
Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/e7e6948c-829b-4d59-ae09-3f991dfe52cf/deployments/90feae8c-ad4f-4f78-83d5-1d2e81b9c578/online


## Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [103]:
payload_data = subscription.payload_logging.get_table_content(limit=60)
payload_data.filter(items=['scoring_id', 'predictedLabel', 'probability'])

,scoring_id,predictedLabel,probability
0,ca54a6f75de6076285158ea757865bbc-56,NOT_BUY,"[0.19033252706754236, 0.8096674729324576]"
1,ca54a6f75de6076285158ea757865bbc-59,BUY,"[0.6569605159222427, 0.34303948407775736]"
2,ca54a6f75de6076285158ea757865bbc-53,BUY,"[0.9077789626849517, 0.09222103731504824]"
3,ca54a6f75de6076285158ea757865bbc-55,BUY,"[0.7425277881749242, 0.25747221182507574]"
4,ca54a6f75de6076285158ea757865bbc-57,NOT_BUY,"[0.27663232213301037, 0.7233676778669895]"
5,ca54a6f75de6076285158ea757865bbc-58,NOT_BUY,"[0.4541249736694374, 0.5458750263305626]"
6,ca54a6f75de6076285158ea757865bbc-51,NOT_BUY,"[0.27945048639686343, 0.7205495136031365]"
7,ca54a6f75de6076285158ea757865bbc-52,NOT_BUY,"[0.09938982373004679, 0.9006101762699533]"
8,ca54a6f75de6076285158ea757865bbc-54,BUY,"[0.8601847145361962, 0.13981528546380376]"
9,ca54a6f75de6076285158ea757865bbc-1,BUY,"[0.8401224994165754, 0.15987750058342476]"


## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the Propensity to Buy model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

## Next steps

OpenScale shows model performance over time. You have two options to keep data flowing to your OpenScale graphs:
  * Download, configure and schedule the [model feed notebook](https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_scoring_feed.ipynb). This notebook can be set up with your WML credentials, and scheduled to provide a consistent flow of scoring requests to your model, which will appear in your OpenScale monitors.
  * Re-run this notebook. Running this notebook from the beginning will delete and re-create the model and deployment, and re-create the historical data. Please note that the payload and measurement logs for the previous deployment will continue to be stored in your datamart, and cal be deleted if necessary.